# PROYECTO: EDA con Python
## Entrega correspondiente al módulo "Python for data" del *Máster en Data & Analytics*

### 1. Importación y Configuración Inicial
Configuramos las librerías principales y ajustes necesarios para el análisis.

In [ ]:
# 1. Librerías principales
import io
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 2. Configuración visual
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
%matplotlib inline 
sns.set_style('whitegrid')
print("Configuraciones visuales básicas aplicadas")

# 3. Configuración de imports locales
import os
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Ruta añadida: {project_root}")

# 4. Módulos propios
try:
    import src.analisis_exploratorio as ae
    import src.data_cleaning as dc
    import src.cleaning_campaing as cc
    print("Módulos importados: analisis_exploratorio, data_cleaning, cleaning_campaing")
except ImportError as e:
    print(f"Error en imports: {e}")

In [ ]:
# =============================================================================
# 5. UTILIDADES PERSONALIZADAS
# =============================================================================
# Este bloque define clases y funciones auxiliares reutilizables específicas
# para el análisis exploratorio de datos en notebooks Jupyter.
# Se mantienen aquí (en lugar de en módulos externos) por las siguientes razones:
# 1. Dependencia del contexto del notebook (io.StringIO, display)
# 2. Uso exclusivo en análisis interactivo (no en producción)
# 3. Facilita la portabilidad del notebook como documento autocontenido
# =============================================================================

class DualOutput:
    """
    Clase para capturar y persistir simultáneamente la salida de análisis EDA.
    
    **PROPÓSITO:**
    Resolver el problema de truncamiento de output en notebooks Jupyter cuando
    el análisis genera texto extenso (>10,000 líneas). Permite:
    - Ver progreso en tiempo real en el notebook
    - Guardar resultado completo en archivo de texto plano
    - Medir tiempo de ejecución del análisis
    
    **ARQUITECTURA:**
    - `self.buffer`: StringIO que actúa como archivo en memoria
    - `self.filepath`: Ruta destino para persistencia
    - `self.start_time`: Timestamp para métricas de rendimiento
    
    **JUSTIFICACIÓN DE UBICACIÓN:**
    No se mueve a src/ porque:
    1. Usa `print()` nativo de Jupyter (comportamiento diferente a logging estándar)
    2. Gestiona memoria con StringIO (específico de entornos interactivos)
    3. No es reutilizable en scripts batch o producción
    4. Forma parte de la "configuración del entorno de análisis"
    
    **USO TÍPICO:**
    >>> output = DualOutput('../data/processed/analisis_completo.txt')
    >>> output.print("=== ANÁLISIS INICIADO ===")
    >>> # ... código de análisis ...
    >>> output.save()  # Persiste todo el buffer en disco
    
    **ALTERNATIVA DESCARTADA:**
    Se consideró usar logging.FileHandler, pero:
    - No permite capturar print() de funciones de terceros
    - Requiere configuración compleja para dual output
    - No proporciona métricas de tamaño de buffer
    """
    
    def __init__(self, filepath: str):
        """
        Inicializa el sistema de captura dual.
        
        Args:
            filepath (str): Ruta absoluta o relativa al archivo destino.
                           Se sobrescribe si existe.
        
        Atributos creados:
            self.filepath: Almacena la ruta para uso posterior
            self.buffer: Buffer en memoria (io.StringIO) para acumular texto
            self.start_time: Marca temporal para cálculo de duración
        
        Ejemplo:
            >>> output = DualOutput('../reports/analisis_2024.txt')
            >>> # Buffer inicializado vacío, listo para recibir datos
        """
        self.filepath = filepath  # Ruta destino (puede contener carpetas)
        self.buffer = io.StringIO()  # Archivo virtual en RAM
        self.start_time = datetime.now()  # T0 para métricas
    
    def print(self, mensaje: str = ""):
        """
        Imprime mensaje en consola Y lo guarda en buffer simultáneamente.
        
        **MECÁNICA INTERNA:**
        1. `print(mensaje)` → Output visible en celda de Jupyter
        2. `self.buffer.write()` → Copia a memoria para guardado posterior
        3. Añade '\n' automáticamente para mantener formato
        
        Args:
            mensaje (str): Texto a imprimir. Acepta strings vacíos para 
                          insertar líneas en blanco.
        
        **CONVERSIÓN DE TIPOS:**
        Usa `str(mensaje)` para garantizar compatibilidad con:
        - DataFrames (repr automático)
        - Números (formateo toString)
        - Objetos personalizados (usa __str__)
        
        Ejemplo:
            >>> output.print("Iniciando análisis...")
            >>> df_stats = pd.DataFrame(...)
            >>> output.print(df_stats)  # Convierte DF a string tabla
            >>> output.print("")  # Línea vacía
        """
        print(mensaje)  # Output interactivo (notebook)
        self.buffer.write(str(mensaje) + '\n')  # Persistencia (archivo)
    
    def save(self):
        """
        Persiste el contenido del buffer en disco y muestra métricas.
        
        **FLUJO DE EJECUCIÓN:**
        1. Abre archivo en modo escritura (sobrescribe contenido previo)
        2. Vuelca buffer completo con `getvalue()`
        3. Calcula tiempo transcurrido desde __init__
        4. Estima tamaño del archivo (KB)
        5. Imprime resumen formateado en consola
        
        **MANEJO DE ERRORES:**
        - `FileNotFoundError`: Si la carpeta no existe
        - `PermissionError`: Si el archivo está abierto en otro programa
        - `IOError`: Problemas de escritura en disco
        
        **MÉTRICAS CALCULADAS:**
        - `elapsed`: Segundos desde inicio (float con 2 decimales)
        - `size_kb`: Tamaño en KB (len(buffer) / 1024)
        
        **CODIFICACIÓN:**
        Usa UTF-8 para soportar:
        - Caracteres españoles (ñ, tildes)
        - Símbolos especiales (€, ©, ®)
        - Emojis (✓, ✗)
        
        Ejemplo de output:
            ======================================================================
            ✓ ANÁLISIS GUARDADO EXITOSAMENTE
            ======================================================================
              Archivo: ../data/processed/analisis_demografico_completo.txt
              Tamaño: 15.3 KB
              Tiempo: 2.47s
            ======================================================================
        
        Raises:
            Exception: Cualquier error de I/O se captura y muestra en formato legible
        """
        try:
            # Apertura de archivo con gestión automática de recursos (context manager)
            with open(self.filepath, 'w', encoding='utf-8') as f:
                f.write(self.buffer.getvalue())  # Volcado completo del buffer
            
            # Cálculo de métricas post-escritura
            elapsed = (datetime.now() - self.start_time).total_seconds()
            size_kb = len(self.buffer.getvalue()) / 1024  # Bytes → KB
            
            # Reporte formateado con símbolos Unicode
            print(f"\n{'='*70}")
            print(f"✓ ANÁLISIS GUARDADO EXITOSAMENTE")
            print(f"{'='*70}")
            print(f"Archivo: {self.filepath}")
            print(f"Tamaño: {size_kb:.1f} KB")
            print(f"Tiempo: {elapsed:.2f}s")
            print(f"{'='*70}")
        
        except Exception as e:
            # Captura genérica para debugging (muestra tipo y mensaje)
            print(f"✗ Error al guardar: {e}")


def guardar_dataframe(df: pd.DataFrame, nombre: str, ruta: str = '../data/processed/') -> str:
    """
    Guarda DataFrame en CSV con logging consistente y validaciones.
    
    **PROPÓSITO:**
    Estandarizar el proceso de guardado de DataFrames procesados con:
    - Nomenclatura uniforme (sin extensión en nombre)
    - Logging detallado de dimensiones y tamaño
    - Retorno de ruta completa para encadenamiento
    
    **JUSTIFICACIÓN DE UBICACIÓN EN NOTEBOOK:**
    Aunque podría estar en src/data_cleaning.py, se mantiene aquí porque:
    1. Usa `print()` directo (compatible con output de celdas)
    2. Formato de logging específico para notebooks interactivos
    3. Parámetros por defecto optimizados para estructura del proyecto
    4. Se usa exclusivamente en fase exploratoria (no en pipelines)
    
    Args:
        df (pd.DataFrame): DataFrame a persistir. Debe tener columnas válidas.
        nombre (str): Nombre base del archivo SIN extensión (.csv se añade auto).
                     Ejemplo: 'df_campaign_clean' → 'df_campaign_clean.csv'
        ruta (str): Directorio destino relativo al notebook. Por defecto '../data/processed/'
                    que asume estructura: notebooks/ y data/processed/ al mismo nivel.
    
    Returns:
        str: Ruta completa del archivo guardado. Útil para logging o verificación.
             Formato: '../data/processed/nombre.csv'
    
    **COMPORTAMIENTO:**
    1. Construye filepath completo (ruta + nombre + '.csv')
    2. Guarda sin índice (index=False) para evitar columnas fantasma
    3. Extrae dimensiones del DataFrame
    4. Calcula tamaño del archivo en disco (os.path.getsize)
    5. Imprime resumen formateado con separador de miles
    
    **FORMATO DE OUTPUT:**
    ✓ DataFrame guardado: df_perfil_cliente.csv
      • Dimensiones: 43,000 filas × 30 columnas
      • Tamaño: 5,247.8 KB
      • Ruta: ../data/processed/df_perfil_cliente.csv
    
    **CONSIDERACIONES:**
    - No valida si la carpeta existe (puede lanzar FileNotFoundError)
    - Sobrescribe archivo si ya existe (sin confirmación)
    - Usa encoding por defecto del sistema (normalmente UTF-8 en sistemas modernos)
    
    **USO TÍPICO:**
    >>> ruta_guardada = guardar_dataframe(df_clean, 'datos_limpios')
    >>> # Continuar con análisis sabiendo que datos están persistidos
    
    **ALTERNATIVA RECHAZADA:**
    No se usa df.to_csv() directamente porque:
    - Requiere recordar index=False cada vez
    - No proporciona feedback visual del tamaño
    - No retorna ruta para validación
    
    Ejemplo completo:
        >>> df_processed = cleaning_pipeline(df_raw)
        >>> archivo = guardar_dataframe(df_processed, 'dataset_final')
        >>> print(f"Datos listos en: {archivo}")
    """
    # Construcción de ruta completa
    filepath = f"{ruta}{nombre}.csv"
    
    # Guardado sin índice (estándar para datasets procesados)
    df.to_csv(filepath, index=False)
    
    # Extracción de metadatos del DataFrame
    filas, cols = df.shape
    
    # Cálculo de tamaño del archivo en disco (post-escritura)
    size_kb = os.path.getsize(filepath) / 1024
    
    # Logging formateado con separador de miles en filas
    print(f"✓ DataFrame guardado: {nombre}.csv")
    print(f"  • Dimensiones: {filas:,} filas × {cols} columnas")
    print(f"  • Tamaño: {size_kb:.1f} KB")
    print(f"  • Ruta: {filepath}\n")
    
    # Retorno de ruta para encadenamiento o validación
    return filepath


# =============================================================================
# CONFIRMACIÓN DE CARGA
# =============================================================================
# Este print final confirma que la celda se ejecutó correctamente
# Útil para debugging si el kernel se reinicia
print("\n✓ Utilidades personalizadas cargadas: DualOutput, guardar_dataframe")

### 2. Carga y Preparación de Datos
Cargamos los datasets de campaña y detalles de clientes.

In [ ]:
# Carga de datos
try:
    # Campaign data
    df_campaign = pd.read_csv('../data/raw/bank-additional.csv', index_col=0)
    
    # Customer details por año
    df_customer_2012 = pd.read_excel(
        '../data/raw/customer-details.xlsx', 
        sheet_name='2012', 
        index_col=0
    )
    df_customer_2013 = pd.read_excel(
        '../data/raw/customer-details.xlsx',
        sheet_name='2013', 
        index_col=0
    )
    df_customer_2014 = pd.read_excel(
        '../data/raw/customer-details.xlsx',
        sheet_name='2014', 
        index_col=0
    )
    print("Datos cargados correctamente")
    
    # Verificación rápida
    print("\nDimensiones:")
    print(f"Campaign: {df_campaign.shape}")
    print(f"Customers 2012-2014: {df_customer_2012.shape}, {df_customer_2013.shape}, {df_customer_2014.shape}")
    
except FileNotFoundError as e:
    print(f"Error cargando datos: {e}")

### 3. Análisis Exploratorio inicial
Análisis sistemático de ambos conjuntos de datos usando nuestros módulos personalizados.

A.- DataFrame de campaña bancaria.

In [ ]:
# Vista previa de los datos de campaña originales
display(df_campaign.head(3))
# Ejemplo
print(df_campaign.dtypes)

### Análisis Inicial: Dataset df_campaign

El dataset original de campaña bancaria presenta múltiples problemas de calidad que requieren tratamiento sistemático.
- Se identificaron variables numéricas (cons.price.idx, cons.conf.idx, euribor3m, nr.employed) incorrectamente tipadas como object, campo age almacenado como float con valores faltantes, campo date en formato texto, variable objetivo y como texto (yes/no) y marcador especial pdays=999 para indicar ausencia de contacto previo.
- Los valores faltantes más significativos afectan a default (21%), euribor3m (22%), age (12%), education (4%) y housing/loan (2%). La nomenclatura es inconsistente (id_, cons.price.idx) y se detectaron columnas latitude/longitude sin justificación analítica clara. 
- El proceso de limpieza debe abordar corrección de tipos, imputación de faltantes, recodificación del target y marcadores especiales, normalización de nomenclatura según PEP 8, optimización de variables categóricas y eliminación de datos irrelevantes.

> Nota: Para detalles completos del proceso de limpieza, consultar el [informe_preliminar.md](../reports/informe_preliminar.md) del proyecto.


In [ ]:
# Limpieza de df_campaign utilizando el módulo de cleaning_campaing diseñado especialmente para este dataset tras haber realizado un primer analisis exploratorio.
df_campaign_clean = cc.clean_campaign_df(df_campaign)
print("Módulo cleaning_campaing aplicado correctamente")

# Normalización de nombres de columnas según estándar PEP 8 (snake_case)
df_campaign_clean = dc.clean_column_names(df_campaign_clean, verbose=True)

# Impresión de confirmación de limpieza para facilitar debugging
print("\nLimpieza de df_campaign completada.\n")

# Vista previa de los datos de campaña limpios y normalizados
display(df_campaign_clean.head(3))
print(df_campaign_clean.dtypes)


In [ ]:
# Guardado del DataFrame df_campaign_clean tras comprobar su limpieza
df_campaign_clean.to_csv('../data/processed/df_campaign_clean.csv', index=False)
print("DataFrame de campaña limpio guardado en '../data/processed/df_campaign_clean.csv'\n")

In [ ]:
# Análisis post-limpieza. Primeras conclusiones.
resultados_campaign_clean = dc.run_checks(
    df_campaign_clean,
    posibles_cat=['education', 'marital', 'default', 'housing', 'loan', 'contact', 'poutcome'],
    call_analisis=True
)

### CONCLUSIONES POST-LIMPIEZA: Dataset df_campaign_clean

El proceso de limpieza y normalización ha transformado el dataset original de campaña bancaria 
en un conjunto de datos de alta calidad con 43,000 registros y 24 variables correctamente tipadas. 

Se han aplicado estándares PEP 8 para nomenclatura (snake_case), convertido variables categóricas 
al tipo category para optimización de memoria, normalizado indicadores económicos, y recodificado 
el target binario (y) de texto a numérico (0/1).

Las variables temporales están en formato datetime64[ns], 
permitiendo análisis de estacionalidad, y se ha creado una bandera previous_contact para tratar el 
marcador especial pdays=999.

El DataFrame resultante permite concentrarse directamente en la extracción de insights.
> Nota: Este resumen resume los principales hallazgos y sugiere enfoques inmediatos para explotar el dataset. Para detalles completos, consultar el [informe_preliminar.md](../reports/informe_preliminar.md) del proyecto 

B. Dataframe Detalles de Clientes.

In [ ]:
# 2. Preparación y análisis de Customer Details
# Combinación de los DataFrames detalles de clientes 2012, 2013, 2014 en un único DataFrame
# Indicamos el año correspondiente a cada DataFrame como clave para mantener la referencia temporal
df_customer_years = pd.concat(
    [df_customer_2012, df_customer_2013, df_customer_2014],
    keys=['2012', '2013', '2014'],
    axis=0
)
# Reseteo de índice para análisis y conversión de la clave año en columna
df_customer_details = df_customer_years.reset_index(level=0)
df_customer_details.rename(columns={'level_0': 'year'}, inplace=True)

# Normalización de nombres de columnas según estándar PEP 8 (snake_case)
df_customer_details = dc.clean_column_names(df_customer_details, verbose=True)

print('DataFrames combinados y normalizados correctamente\n')

In [ ]:
# Normalización manual adicional de la columna numwebvisitsmonth a num_web_visits_month
df_customer_details.rename(columns={'numwebvisitsmonth': 'num_web_visits_month'}, inplace=True)
print("Columna 'numwebvisitsmonth' renombrada a 'num_web_visits_month' correctamente\n")

In [ ]:
# Vista previa de los datos de detalles de clientes combinados.
display(df_customer_details.sample(3))
print(df_customer_details.dtypes)

In [ ]:
# Guardado del DataFrame combinado y normalizado.
df_customer_details.to_csv('../data/processed/df_customer_details.csv', index=False)
print("DataFrame detalles de clientes guardado en '../data/processed/df_customer_details.csv'\n")

In [ ]:
# Análisis post-limpieza de detalles de clientes. Primeras conclusiones.
resultados_customers = dc.run_checks(
    df_customer_details,
    posibles_cat=['year'],  # Añadir otras categóricas detectadas
    call_analisis=True
)

### CONCLUSIONES POST-LIMPIEZA: Dataset df_customer_details

El DataFrame df_customer_details integra registros de 2012-2014 totalizando 43,170 observaciones con 7 variables correctamente tipadas bajo nomenclatura PEP 8 (snake_case), incluyendo corrección manual de num_web_visits_month.

Las variables temporales están en formato datetime64[ns], year convertida a category para optimización, y numéricas en int64. A diferencia del dataset de campaña, no presenta valores faltantes ni inconsistencias de formato confirmando origen transaccional robusto.

Los identificadores únicos garantizan uniones fiables con df_campaign_clean y las distribuciones de income, kidhome, teenhome y num_web_visits_month son coherentes con perfiles bancarios reales.

La concentración desigual por año (mayor en 2012) debe considerarse en análisis temporales para evitar sesgos de representatividad.

>Nota: Este resumen resume los principales hallazgos y sugiere enfoques inmediatos para explotar el dataset. Para detalles completos, consultar el [informe_preliminar.md](../reports/informe_preliminar.md) del proyecto 

### Unión de los datasets listos para su modelado.

Combinamos ambos DF procesados para obtener un Conjunto de Datos de Perfil Completo del Cliente (*Master Dataset*) de Marketing Bancario que nos permita predecir la variable objetivo (y), utilizando toda la información disponible: las características fijas del cliente, las variables de campaña y los indicadores económicos.

In [ ]:
# 1. Unir los DataFrames y fijar 'id' como índice del DataFrame resultante
df_perfil_cliente = pd.merge(
    df_campaign_clean,       # Tabla izquierda (campaña)
    df_customer_details,     # Tabla derecha (detalles cliente)
    left_on='id',
    right_on='id',
    how='inner'
)
df_perfil_cliente.index.name = None
df_perfil_cliente.set_index('id', inplace=True)
df_perfil_cliente.index.name = None

print('Combinación de DataFrames completada. Índice establecido en "id".')
display(df_perfil_cliente.sample(3))
print(df_perfil_cliente.dtypes)


In [ ]:
# Guardado del DataFrame final para EDA, modelado y visualización
df_perfil_cliente.to_csv('../data/processed/df_perfil_cliente.csv', index=True,)
print("DataFrame final guardado en '../data/processed/df_perfil_cliente.csv'")

# EDA
**Analisis Exploratorio de Datos. Perfil de Cliente**
###

## 1- Perfil Demográfico (Income y Antigüedad)

## Análisis Demográfico Completo

**Objetivo:** Identificar patrones diferenciadores entre clientes que suscriben (y=1) vs. rechazan (y=0).

**Variables analizadas:**
- Numéricas: `income`, `antiguedad_años`, `age`
- Categóricas: `segmento_edad`, `education`, `job`, `marital`

**Output:** [`analisis_demografico_completo.txt`](../reports/outputs/analisis_demografico_completo.txt)

---

In [45]:
# Inicializar sistema de logging dual
os.makedirs('../reports/outputs', exist_ok=True)
output = DualOutput('../reports/outputs/analisis_demografico_completo.txt')
print("Sistema de logging dual inicializado correctamente\n")
# Asegurar tipos mínimos
df_perfil_cliente['y'] = pd.to_numeric(
    df_perfil_cliente['y'].replace({'yes':1, 'no':0, 'si':1, 'sí':1}),
    errors='coerce'
).astype('Int8')
print("Columna 'y' convertida a tipo Int8 correctamente\n")

# Crear segmento_edad si falta y fijar orden
if 'segmento_edad' not in df_perfil_cliente.columns:
    df_perfil_cliente['segmento_edad'] = pd.cut(
        pd.to_numeric(df_perfil_cliente['age'], errors='coerce'),
        bins=[0, 25, 35, 45, 55, 65, 120],
        labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
    )
orden_edad = ['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
df_perfil_cliente['segmento_edad'] = pd.Categorical(
    df_perfil_cliente['segmento_edad'], categories=orden_edad, ordered=True
)
print("Columna 'segmento_edad' creada y ordenada correctamente\n")

In [ ]:
# =============================================================================
# 1. ANÁLISIS DE VARIABLES NUMÉRICAS
# =============================================================================
variables_numericas = {
    'income': {'titulo': 'INGRESOS DEL CLIENTE', 'formato_ratio': '€{:.0f}'},
    'antiguedad_años': {'titulo': 'ANTIGÜEDAD DEL CLIENTE', 'formato_ratio': '{:.2f} años'},
    'age': {'titulo': 'EDAD DEL CLIENTE', 'formato_ratio': '{:.1f} años'}
}

# Calcular antigüedad en años si no existe
if 'antiguedad_años' not in df_perfil_cliente.columns:
    df_perfil_cliente['antiguedad_dias'] = (
        pd.to_datetime(df_perfil_cliente['date'], errors='coerce') -
        pd.to_datetime(df_perfil_cliente['dt_customer'], errors='coerce')
    ).dt.days
    df_perfil_cliente['antiguedad_años'] = df_perfil_cliente['antiguedad_dias'] / 365.25

for idx, (variable, config) in enumerate(variables_numericas.items(), 1):
    print(f"✓ Analizando {idx}/{len(variables_numericas)}: {config['titulo']}...")
    output.print("\n" + "="*70)
    output.print(f"ANÁLISIS DEMOGRÁFICO: {config['titulo']}")
    output.print("="*70)

    serie_num = pd.to_numeric(df_perfil_cliente[variable], errors='coerce')
    stats = serie_num.groupby(df_perfil_cliente['y']).agg(['mean','median','std','count'])
    output.print(f"\n--- Estadísticas de {config['titulo']} por Resultado (y) ---")
    output.print(str(stats.round(2)))

    # ✅ FIX: Conversión explícita a float con validación de tipo
    try:
        mean_y0 = float(stats.loc[0, 'mean']) if 0 in stats.index else np.nan # type: ignore 
        mean_y1 = float(stats.loc[1, 'mean']) if 1 in stats.index else np.nan # type: ignore
        
        # Validar que ambos valores sean finitos y no-NaN
        if pd.notna(mean_y0) and pd.notna(mean_y1) and mean_y0 != 0:
            ratio = mean_y1 / mean_y0
            diff = mean_y1 - mean_y0
            
            output.print("\nMétricas Comparativas:")
            output.print(f"  • Ratio (y=1 / y=0): {ratio:.3f}x")
            output.print(f"  • Diferencia: {config['formato_ratio'].format(diff)}")
            
            if ratio > 1.1:
                output.print(f"  → Clientes exitosos tienen {(ratio-1)*100:.1f}% más {variable}")
            elif ratio < 0.9:
                output.print(f"  → Clientes exitosos tienen {(1-ratio)*100:.1f}% menos {variable}")
        else:
            output.print("\n⚠ No se calcularon comparativas (clase faltante, NaN o división por cero)")
    
    except (TypeError, ValueError) as e:
        output.print(f"\n⚠ Error en conversión de tipos: {e}")

In [ ]:

# =============================================================================
# 2. ANÁLISIS DE VARIABLES CATEGÓRICAS
# =============================================================================
print("\n✓ Iniciando análisis de VARIABLES CATEGÓRICAS...")
tasas_demograficas = {}

# 2.1 SEGMENTO DE EDAD (ordinal)
t_edad = ae.calcular_tasa_proporciones(df_perfil_cliente, 'segmento_edad').sort_values(
    'segmento_edad', key=lambda s: s.map({k:i for i,k in enumerate(orden_edad)})
)
tasas_demograficas['segmento_edad'] = t_edad
output.print("\n" + "="*70)
output.print("ANÁLISIS DEMOGRÁFICO: SEGMENTO DE EDAD")
output.print("="*70)
output.print("\n--- Tasa de Suscripción por SEGMENTO DE EDAD ---")
output.print(str(t_edad))

# 2.2 EDUCATION (ordinal)
map_edu = {
    'illiterate':0,'basic.4y':1,'basic.6y':2,'basic.9y':3,
    'high.school':4,'professional.course':5,'university.degree':6
}
df_tmp = df_perfil_cliente.copy()
df_tmp['education_ord'] = df_tmp['education'].map(map_edu)
t_edu = ae.calcular_tasa_proporciones(df_tmp, 'education')
tasas_demograficas['education'] = t_edu
output.print("\n" + "="*70)
output.print("ANÁLISIS DEMOGRÁFICO: NIVEL EDUCATIVO")
output.print("="*70)
output.print("\n--- Tasa de Suscripción por NIVEL EDUCATIVO ---")
output.print(str(t_edu.sort_values('tasa_exito', ascending=False)))

# 2.3 JOB (nominal; combinar clases raras y mostrar Top/Bottom)
freq = df_perfil_cliente['job'].value_counts(normalize=True)
rare = freq[freq < 0.01].index
df_job = df_perfil_cliente.copy()
df_job.loc[df_job['job'].isin(rare), 'job'] = 'other'

t_job = ae.calcular_tasa_proporciones(df_job, 'job').sort_values('tasa_exito', ascending=False)
tasas_demograficas['job'] = t_job
output.print("\n" + "="*70)
output.print("ANÁLISIS DEMOGRÁFICO: OCUPACIÓN/PROFESIÓN")
output.print("="*70)
output.print("\n--- Tasa de Suscripción por OCUPACIÓN/PROFESIÓN ---")
output.print(str(t_job))
top_n = 5
output.print(f"\n--- Top {top_n} OCUPACIÓN/PROFESIÓN (Mayor Conversión) ---")
output.print(str(t_job.head(top_n)[['job','tasa_exito','total']]))
output.print(f"\n--- Bottom {top_n} OCUPACIÓN/PROFESIÓN (Menor Conversión) ---")
output.print(str(t_job.tail(top_n)[['job','tasa_exito','total']]))

# 3.4 MARITAL (nominal; baja cardinalidad)
t_marital = ae.calcular_tasa_proporciones(df_perfil_cliente, 'marital').sort_values('tasa_exito', ascending=False)
tasas_demograficas['marital'] = t_marital
output.print("\n" + "="*70)
output.print("ANÁLISIS DEMOGRÁFICO: ESTADO CIVIL")
output.print("="*70)
output.print("\n--- Tasa de Suscripción por ESTADO CIVIL ---")
output.print(str(t_marital))


In [ ]:
# =============================================================================
# 4. RESUMEN FINAL Y GUARDADO
# =============================================================================
output.print("\n" + "="*70)
output.print("RESUMEN EJECUTIVO: PERFIL DEMOGRÁFICO")
output.print("="*70)

# Variables numéricas: mostrar top insights
output.print("\n📊 VARIABLES NUMÉRICAS - Diferencias clave (y=1 vs y=0):")
for var, config in variables_numericas.items():
    serie = pd.to_numeric(df_perfil_cliente[var], errors='coerce')
    stats = serie.groupby(df_perfil_cliente['y']).agg(['mean'])
    if 0 in stats.index and 1 in stats.index:
        m0, m1 = stats.loc[0, 'mean'], stats.loc[1, 'mean']
        if m0 != 0:
            ratio = float(m1) / float(m0)  # type: ignore[operator]
            output.print(f"  • {config['titulo']}: {ratio:.2f}x (y=1/y=0)")

# Variables categóricas: mostrar tasas extremas
output.print("\n📊 VARIABLES CATEGÓRICAS - Tasas de conversión destacadas:")
for var_name, df_tasas in tasas_demograficas.items():
    if not df_tasas.empty:
        top = df_tasas.nlargest(1, 'tasa_exito')
        bot = df_tasas.nsmallest(1, 'tasa_exito')
        output.print(f"\n  {var_name.upper()}:")
        output.print(f"    Mayor: {top.iloc[0][var_name]} → {top.iloc[0]['tasa_exito']:.1%}")
        output.print(f"    Menor: {bot.iloc[0][var_name]} → {bot.iloc[0]['tasa_exito']:.1%}")

# Guardar archivo y mostrar métricas
output.save()

print("\n✓ Análisis demográfico completo finalizado")
# ...existing code...

In [ ]:
# VERIFICACIÓN DE CONFIGURACIÓN
print("="*70)
print("VERIFICACIÓN DEL ENTORNO")
print("="*70)

# 1. Verificar imports
try:
    assert 'DualOutput' in dir()
    print("✓ Clase DualOutput disponible")
except:
    print("❌ DualOutput no encontrada - revisar celda de imports")

# 2. Verificar módulos
try:
    assert hasattr(ae, 'calcular_tasa_proporciones')
    print("✓ Función calcular_tasa_proporciones disponible")
except:
    print("❌ Módulo analisis_exploratorio no cargado correctamente")

# 3. Verificar columnas derivadas
columnas_requeridas = ['antiguedad_años', 'segmento_edad']
for col in columnas_requeridas:
    if col in df_perfil_cliente.columns:
        print(f"✓ Columna '{col}' creada")
    else:
        print(f"⚠️  Columna '{col}' falta - ejecutar análisis demográfico")

# 4. Verificar tasas
if 'tasas_demograficas' in dir():
    print(f"✓ Tasas calculadas para {len(tasas_demograficas)} variables")
else:
    print("⚠️  Variable 'tasas_demograficas' no existe")

print("\n" + "="*70)

Factores de Campaña (duration, poutcome)

In [ ]:
'''
# --- 1. Duración de la Llamada (duration) ---
print("\n---  Duración Promedio (Segundos) por Resultado de Campaña (y) ---")
duration_stats = df_perfil_cliente.groupby('y')['duration'].agg(['mean', 'median', 'count'])
print(duration_stats.round(0))

#  Nota sobre la Duración: Es un 'data leakage'. Un modelo lo usaría como atajo.
print("\n Advertencia: 'duration' es una variable de filtrado (data leakage). Muestra la relación, pero debe usarse con cautela en el modelado predictivo.")

# Visualización con Boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x='y', y='duration', data=df_perfil_cliente, palette='rocket', hue='y', legend=False)
plt.title('Distribución de Duración de Llamada vs. Suscripción')
plt.xlabel('Suscripción (0=No, 1=Sí)')
plt.ylabel('Duración (Segundos)')
plt.show()

# --- 2. Resultado de Campaña Anterior (poutcome) ---
ae.calcular_tasa_proporciones(df_perfil_cliente, 'poutcome')

'''

In [ ]:
'''
# Filtrar solo a los clientes que realizaron la venta (y=1)
df_exito = df_perfil_cliente[df_perfil_cliente['y'] == 1].copy()

# Analizar la media y moda del perfil de éxito:
# Ejemplos de análisis:
# Ingreso Promedio: df_exito['Income'].mean()
# Edad Promedio: df_exito['age'].mean()
# Ocupación más Común: df_exito['job'].mode()[0]
results = dc.run_checks(df_exito, 
                       posibles_cat=['education', 'marital'],
                       call_analisis=True)
df_processed = results['df']

'''

El análisis preliminar identifica variables clave para modelado predictivo con diferentes niveles 
de prioridad: máxima para el historial de campaña (poutcome, previous_contact, campaign) y el perfil 
demográfico (age, job, education), alta para el contexto económico (emp_var_rate, euribor3m, cons_price_idx), 
y media para variables de situación financiera (default, housing, loan) y temporales (contact_month). 
Se han detectado alertas críticas que requieren atención: verificar el desbalance de clases en la 
variable objetivo y, confirmar multicolinealidad entre variables económicas, evaluar el impacto de 
estacionalidad mensual, y excluir duration del modelado por data leakage confirmado (variable solo 
conocida después del resultado). El dataset alcanza una calificación de 9/10 en calidad y está listo 
para análisis bivariado, detección de outliers, y construcción de modelos robustos.
